In [17]:
library(abind)
library(IRdisplay)
if (getwd() != "/projects/ps-gymreklab/amassara/simulate_gwas") {
    setwd("/projects/ps-gymreklab/amassara/simulate_gwas")
}

In [91]:
gt = "out/1_98001984-99001984/gt_matrix.tsv.gz"
phen = "out/1_98001984-99001984/phens.tsv.gz"
out = "out/1_98001984-99001984/susieR"
dir.create(out, showWarnings = FALSE)

In [92]:
# import genotype matrices as proper matrices
gt = read.csv(gt, sep="\t", header=T)
phen = read.csv(phen, sep="\t", header=T)
# the number of samples and the number of variants:
n = nrow(gt)
p = ncol(gt)
# create matrices without unecessary columns
X = as.matrix(gt[,-1])
storage.mode(X) = 'double'
y = as.matrix(phen[,ncol(phen)])

In [93]:
# create vector with causal status
# this vector indicates which variant is truly causal
b = rep(0,p)
names(b) = colnames(X)
b['X98332868.0'] = 1 # SNP
# b['X98506615.1'] = 1 # STR

In [94]:
mm_regression = function(X, Y, Z=NULL) {
  if (!is.null(Z)) {
      Z = as.matrix(Z)
  }
  reg = lapply(seq_len(ncol(Y)), function (i) simplify2array(susieR:::univariate_regression(X, Y[,i], Z)))
  reg = do.call(abind, c(reg, list(along=0)))
  # return array:
  #   out[1,,] is beta hat (the least-squares estimates of the coefficients)
  #   out[2,,] is se betahat (the standard errors of the beta hats)
  return(aperm(reg, c(3,2,1)))
}
sumstats = mm_regression(as.matrix(X), as.matrix(y))
dat = list(X=X,Y=as.matrix(y))
input = paste0(out,'/sumstats.rds')
saveRDS(list(data=dat, sumstats=sumstats), input)

In [95]:
output = paste0(out, "/N2finemapping.FINEMAP")
args = "--n-causal-snps 2"
commandArgs = function(...) 1

# source(paste0(.libPaths(), '/susieR/code/finemap.R'))
source("/projects/ps-gymreklab/amassara/simulate_gwas/temp/finemap.R")

Warning message in cor(dat$data$X):
“the standard deviation is zero”


In [96]:
finemap = readRDS(paste0(out,"/N2finemapping.FINEMAP.rds"))[[1]]
head(finemap$set)

,rank,config,prob,log10bf,odds,k,prob_norm_k,h2,h2_0.95CI,mean,sd
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,1,5792,0.0556198,2.60940,1.00000,1,0.0573614,0.0249327,"0.00647495,0.0549603",NA,NA
2,2,3770,0.0262347,2.28304,2.12008,1,0.0270562,0.0226365,"0.00475097,0.0482768",NA,NA
3,3,7898,0.0237931,2.24062,2.33764,1,0.0245381,0.0223360,"0.00478728,0.0524536",NA,NA
4,4,7072,0.0237931,2.24062,2.33764,1,0.0245381,0.0223360,"0.00500782,0.0513357",NA,NA
5,5,1743,0.0185989,2.13365,2.99049,1,0.0191813,0.0215761,"0.00331881,0.0483662",NA,NA
6,6,1727,0.0139863,2.00987,3.97674,1,0.0144242,0.0206926,"0.00399424,0.0462061",NA,NA


In [97]:
snp = finemap$snp
pip = snp[order(as.numeric(snp$snp)),]$snp_prob

In [98]:
pdf(paste0(out,'/finemap.pdf'), width =5, height = 5, pointsize=16)
susieR::susie_plot(pip, y='PIP', b=b, main = 'Bayesian sparse regression')
dev.off()

png 
  2

In [99]:
display_pdf(file=paste0(out,'/finemap.pdf'))

In [100]:
fitted = susieR::susie(X, y, L=5,
               estimate_residual_variance=TRUE, 
               scaled_prior_variance=0.2,
               tol=1e-3, track_fit=TRUE, min_abs_corr=0.02)

In [101]:
pdf(paste0(out,'/susie.pdf'), width =5, height = 5, pointsize=16)
susieR::susie_plot(fitted, y='PIP', b=b, max_cs=0.4, main = paste('SuSiE, ', length(fitted$sets$cs), 'CS identified'))
dev.off()

png 
  2

In [102]:
display_pdf(file=paste0(out,'/susie.pdf'))

## SuSiE Effect Size Estimate

In [103]:
bhat = coef(fitted)[-1]
pdf(paste0(out,'/susie_eff.pdf'), width =5, height = 5, pointsize=16)
susieR::susie_plot(bhat, y='bhat', b=b, main = 'SuSiE, effect size estimate') 
dev.off()

png 
  2

In [104]:
display_pdf(file=paste0(out,'/susie_eff.pdf'))